In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import sqlite3
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# **Scraping Database**

#### **Scraping URLs**

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

headers = {"User-Agent" : "User-Agent:Mozilla/5.0 (Windows NT 10.0;"}
urls =[]
i = 0
year = 2017

while i < 15:
    while year < 2024:
        page=requests.get('https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected='+str(year)+'&distribution=&sort=desc&view=detailed&page='+str(i),headers=headers)
        
        i = i+1

        soup = BeautifulSoup(page.content, 'html.parser')

        game_containers = soup.find_all('td', class_ = 'clamp-summary-wrap')

        for container in game_containers:
            url = container.find('a', attrs={'class': 'title'})['href']
            urls.append('www.metacritic.com'+str(url))
        year = year+1

url_folder = pd.DataFrame({'url': urls})
url_folder.to_csv('urlsall.csv')

### **Scraping Mainframe**

In [ ]:
urls = pd.read_csv('urlsall.csv',header=None)
urls.drop([0],inplace=True)
urls

In [ ]:
game_title = []
game_date = []
critic_score = []
user_score = []
game_platform = []
game_genre = []
review_bodys = []
game_pulisher = []

urllinks = urls[1]

import random
import time
from time import sleep


for each_url in urllinks:
    header= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0"}
    wait_seconds = random.uniform(0, 1)
    time.sleep(wait_seconds)
    page=requests.get('http://'+str(each_url),headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')

    containers_title = soup.find('div', class_ = 'product_title')
    containers_sub = soup.find('div', class_ = 'product_data')
    containers_metascore = soup.find('a', class_ = 'metascore_anchor')
    containers_user = soup.find('div', class_ = 'userscore_wrap feature_userscore')
    containers_details = soup.find('li', class_ = 'summary_detail product_genre')

    #Game Title
    try:
        game_title_text = containers_title.select('.hover_none')[0].get_text().strip()
        game_title.append(game_title_text)
    except:
        game_title.append('')

    #Game Platform
    try:            
        game_platform_text = containers_title.select('.platform')[0].get_text().strip()
        game_platform.append(game_platform_text)
    except:
        game_platform.append('')

    #Critic Score
    try:
        critic_score_text = containers_metascore.find('span',itemprop='ratingValue').get_text()
        critic_score.append(critic_score_text)
    except:
        critic_score.append('')

    #User Score
    try:
        user_score_text = containers_user.find('a', class_='metascore_anchor').get_text().strip()
        user_score.append(user_score_text)
    except:
        user_score.append('')

    #Game Genre
    try:
        game_genre_text = containers_details.find_all('span',class_='data')
        game_genre_temp = []
        for game_genre_i in game_genre_text:
            game_genre_temp.append(game_genre_i.get_text())
        game_genre.append('/'.join(game_genre_temp))
    except:
        game_genre.append('')

    #Game Publisher
    try:
        publisher_text = containers_sub.find('span', class_='data').get_text().strip().replace('  ','').replace('\n','').replace('\t','').replace(', ','/')
        game_pulisher.append(publisher_text)
    except:
        game_pulisher.append('')

    #Game Release Date
    containers_date = soup.find('li', class_ = 'summary_detail release_data')
    try:
        date_text = containers_date.find('span', class_='data').get_text().strip()
        game_date.append(date_text)
    except:
        game_date.append('')
    
    print(str(game_title_text)+str(game_platform_text)+str(critic_score_text)+str(user_score_text)+str(game_genre_text)+str(publisher_text)+str(date_text))

In [ ]:
Dataall = pd.DataFrame({'Game Title':game_title,
                            'Game Publisher':game_pulisher,
                            'Game Platform':game_platform,
                            'Game Genre':game_genre,
                            'Game Release Date':game_date,
                            'Critic Score':critic_score,
                            'User Score':user_score,
                            })
Dataall.dropna(inplace=True)
Dataall['Game Release Date'] = pd.to_datetime(Data2020_2023['Game Release Date'])
Dataall['Year'] = Dataall['Game Release Date'].dt.year
Dataall.dropna(inplace=True)
Dataall.to_csv('Dataall.csv')

## **Setting up Database**

##### **Pandas**

In [2]:
df = pd.read_csv('Dataall.csv')
df = df.drop(df[df['Year'] == 2005].index)
df.columns = df.columns.str.replace(' ', '_')
df['Game_Release_Date'] = pd.to_datetime(df['Game_Release_Date'])
df

,Game_Title,Game_Publisher,Game_Platform,Game_Genre,Game_Release_Date,Critic_Score,User_Score,Year
1,Milkmaid of the Milky Way,machineboy,PC,Adventure/Point-and-Click,2017-01-04,74,7.0,2017.0
2,Hatsune Miku: Project Diva Future Tone - Futur...,Sega,PlayStation 4,Action/Rhythm/Music,2017-01-10,84,7.7,2017.0
3,Hatsune Miku: Project Diva Future Tone - Color...,Sega,PlayStation 4,Action/Rhythm/Music,2017-01-10,84,7.9,2017.0
4,Nevermind,Flying Mollusk,Xbox One,Action Adventure/Adventure/3D/Linear/First-Person,2017-01-11,65,6.3,2017.0
5,Detention,Red Candle Games,PC,Adventure/General,2017-01-12,83,7.8,2017.0
...,...,...,...,...,...,...,...,...
6299,DREDGE,Team17/Fireshine Games,PlayStation 5,Adventure/General,2023-03-30,81,tbd,2023.0
6300,The Last Worker,Wired Productions,PlayStation 5,Adventure/3D/First-Person,2023-03-30,74,tbd,2023.0
6301,DREDGE,Team17/Black Salt Games,PC,Adventure/General,2023-03-30,81,tbd,2023.0
6302,Ravenbound,Systemic Reaction,PC,Role-Playing/Action RPG,2023-03-30,59,tbd,2023.0


## **Creating Functions**

### **Specific Game trend**

In [3]:
def get_specific():
  specific = (input("Enter the trend name: "))
  df_specific = df
  df_specific.columns = df_specific.columns.str.replace(' ', '_')
  df_specificfuzzy = df_specific.query('Game_Publisher.str.contains("'+specific+'")', engine='python')
  df_specificfuzzy = df_specificfuzzy.drop_duplicates(subset=['Game_Publisher'])
  df_specificfuzzy = '/'.join(df_specificfuzzy['Game_Publisher'])
  df_specificfuzzy = df_specificfuzzy.split('/')
  df_specificfuzzy = pd.DataFrame(df_specificfuzzy)
  df_specificfuzzy.columns = ['Game_Publisher']
  df_specificfuzzy = df_specificfuzzy.drop_duplicates(subset=['Game_Publisher'])
  df_specificfuzzy = df_specificfuzzy.drop(df_specificfuzzy[~df_specificfuzzy['Game_Publisher'].str.contains(specific)].index)
  df_specificfuzzy = df_specificfuzzy.sort_values(by=['Game_Publisher']).reset_index(drop=True)
  df_specificfuzzy.index = df_specificfuzzy.index+1
  display(df_specificfuzzy)

  specific_index = (input("Enter the specific index: "))
  specific_value = df_specificfuzzy.loc[int(specific_index), 'Game_Publisher']
  print(specific_value)

  df_specific  = df[df['Game_Publisher'] == specific_value]

  df_specific_criticscore = df_specific .groupby(['Game_Title'])['Critic_Score'].mean().round(2).reset_index()
  df_specific_criticscore.sort_values(by=['Critic_Score'], ascending=False, inplace=True)
  df_specific_critic_show = df_specific_criticscore.set_index('Game_Title')

  df_specific_userscore = df_specific .drop(df_specific [df_specific ['User_Score'] == 'tbd'].index)
  df_specific_userscore = df_specific_userscore.dropna(subset=['User_Score'])
  df_specific_userscore['User_Score'] = df_specific_userscore['User_Score'].astype(float)
  df_specific_userscore = df_specific_userscore.groupby(['Game_Title'])['User_Score'].mean().round(2).reset_index()
  df_specific_userscore.sort_values(by=['User_Score'], ascending=False, inplace=True)
  df_specific_user_show = df_specific_userscore.set_index('Game_Title')

  df_specific_real = pd.merge(df_specific_critic_show, df_specific_user_show, on='Game_Title')
  df_specific_critic_real = df_specific_real.sort_values(by=['Critic_Score'], ascending=True).tail(10)
  df_specific_user_real = df_specific_real.sort_values(by=['User_Score'], ascending=True).tail(10)

  figcriticshow = px.bar(df_specific_critic_real, y=df_specific_critic_real.index, x="Critic_Score", title='Top 10 Games by Critic Score of '+specific_value,orientation='h', text='Critic_Score', color_discrete_sequence=['#2E7D32'])
  figcriticshow.update_traces(texttemplate='%{text:.4s}', textposition='outside')
  figcriticshow.add_trace(go.Scatter(y=df_specific_critic_real.index, x=df_specific_critic_real['User_Score']*10, text='User_Score',mode='lines+markers', orientation = 'h',name='User_Score', marker_color='#FFA000'))
  figcriticshow.show()

  figusershow = px.bar(df_specific_user_real, y=df_specific_user_real.index, x="Critic_Score", title='Top 10 Games by User Score of '+specific_value,orientation='h', color_discrete_sequence=['rgb(179, 229, 252)'])
  figusershow.add_trace(go.Scatter(y=df_specific_user_real.index, x=df_specific_user_real['User_Score']*10, text='User_Score',textposition='middle right',mode='lines+markers', orientation = 'h',name='User_Score', marker_color='rgb(255, 0, 0)'))
  figusershow.show()

  df_specific_compare = df_specific 
  df_specific_compare = df_specific_compare.drop(df_specific_compare[df_specific_compare['User_Score'] == 'tbd'].index)
  df_specific_compare = df_specific_compare.dropna(subset=['User_Score'])
  df_specific_compare['User_Score'] = df_specific_compare['User_Score'].astype(float)
  df_specific_compare = df_specific_compare.groupby(['Year'])['User_Score','Critic_Score'].mean().round(2).reset_index()
  df_specific_compare
  
  figcompare = px.line(df_specific_compare, x="Year", y=["User_Score"], title='User Score specific  by platforms',markers=True)
  figcompare.add_trace(go.Scatter(x=df_specific_compare['Year'], y=df_specific_compare['Critic_Score']/10, name='Critic_Score'))
  figcompare.update_traces(textposition="bottom right")
  figcompare.show()
  
  df_specific_platform = df_specific .groupby(['Game_Platform','Year']).mean().round(2).reset_index()
  df_specific_year = df_specific .groupby(['Year']).mean().round(2).reset_index()
  
  figplat = px.line(df_specific_platform, x="Year", y="Critic_Score", color='Game_Platform',title='Average Critic Score Trend by platforms',markers=True)
  figplat.update_traces(textposition="bottom right")
  figplat.update_yaxes(range=[30, 100])
  figplat.add_trace(go.Bar(
  x=df_specific_year['Year'], y=df_specific_year['Critic_Score'], text=df_specific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplat.show()

  df_specific_platform_user = df_specific .drop(df_specific [df_specific ['User_Score'] == 'tbd'].index)
  df_specific_platform_user = df_specific_platform_user.dropna(subset=['User_Score'])
  df_specific_platform_user['User_Score'] = df_specific_platform_user['User_Score'].astype(float)
  df_specific_platform_user = df_specific_platform_user.groupby(['Game_Platform','Year'])['User_Score'].mean().round(2).reset_index()
  
  figplatuser = px.line(df_specific_platform_user, x="Year", y="User_Score", color='Game_Platform',title='Average User Score Trend by platforms',markers=True)
  figplatuser.update_traces(textposition="bottom right")
  figplatuser.update_yaxes(range=[0, 10])
  figplatuser.add_trace(go.Bar(
  x=df_specific_year['Year'], y=df_specific_year['Critic_Score']/10, text=df_specific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplatuser.show()

  df_specific_genre = df_specific 
  df_specific_genre['Game_Genre'] = df_specific_genre['Game_Genre'].str.split('/')
  df2 = df_specific_genre.explode('Game_Genre').reset_index(drop=True)

  df_specific_gen_user = df2.drop(df2[df2['User_Score'] == 'tbd'].index)
  df_specific_gen_user = df_specific_gen_user.dropna(subset=['User_Score'])
  df_specific_gen_user['User_Score'] = df_specific_gen_user['User_Score'].astype(float)

  df_specific_gen = df2.groupby(['Game_Genre','Year']).mean().round(2).reset_index()
  df_specific_gen_user = df_specific_gen_user.groupby(['Game_Genre','Year'])['User_Score'].mean().round(2).reset_index()

  figplatgen = px.line(df_specific_gen, x="Year", y="Critic_Score", color='Game_Genre',title='Average Critic Score Trend by genres',markers=True)
  figplatgen.update_traces(textposition="bottom right")
  figplatgen.update_yaxes(range=[0, 100])
  figplatgen.add_trace(go.Bar(
  x=df_specific_year['Year'], y=df_specific_year['Critic_Score'], text=df_specific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplatgen.show()

  figplatgen_user = px.line(df_specific_gen_user, x="Year", y="User_Score", color='Game_Genre',title='Average User Score Trend by genres',markers=True)
  figplatgen_user.update_traces(textposition="bottom right")
  figplatgen_user.update_yaxes(range=[0, 10])
  figplatgen_user.add_trace(go.Bar(
  x=df_specific_year['Year'], y=df_specific_year['Critic_Score']/10, text=df_specific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplatgen_user.show()

## **All Time Trend**

In [4]:
def alltimetrend():
  dftrend = df
  dftrend = dftrend.drop(dftrend[dftrend['Year'] == 2005].index)
  dftrend_criticscore = dftrend .groupby(['Game_Title'])['Critic_Score'].mean().round(2).reset_index()
  dftrend_criticscore.sort_values(by=['Critic_Score'], ascending=False, inplace=True)
  dftrend_critic_show = dftrend_criticscore.set_index('Game_Title')

  dftrend_userscore = dftrend .drop(dftrend [dftrend ['User_Score'] == 'tbd'].index)
  dftrend_userscore = dftrend_userscore.dropna(subset=['User_Score'])
  dftrend_userscore['User_Score'] = dftrend_userscore['User_Score'].astype(float)
  dftrend_userscore = dftrend_userscore.groupby(['Game_Title'])['User_Score'].mean().round(2).reset_index()
  dftrend_userscore.sort_values(by=['User_Score'], ascending=False, inplace=True)
  dftrend_user_show = dftrend_userscore.set_index('Game_Title')

  dftrend_real = pd.merge(dftrend_critic_show, dftrend_user_show, on='Game_Title')
  dftrend_critic_real = dftrend_real.sort_values(by=['Critic_Score'], ascending=True).tail(10)
  dftrend_user_real = dftrend_real.sort_values(by=['User_Score'], ascending=True).tail(10)

  figcriticshow = px.bar(dftrend_critic_real, y=dftrend_critic_real.index, x="Critic_Score", title='Top 10 Games by Critic Score',orientation='h', text='Critic_Score', color_discrete_sequence=['#2E7D32'])
  figcriticshow.update_traces(texttemplate='%{text:.4s}', textposition='outside')
  figcriticshow.add_trace(go.Scatter(y=dftrend_critic_real.index, x=dftrend_critic_real['User_Score']*10, text='User_Score',mode='lines+markers', orientation = 'h',name='User_Score', marker_color='#FFA000'))
  figcriticshow.update_xaxes(range=[50, 100])
  figcriticshow.show()

  figusershow = px.bar(dftrend_user_real, y=dftrend_user_real.index, x="Critic_Score", title='Top 10 Games by User Score',orientation='h', color_discrete_sequence=['rgb(179, 229, 252)'])
  figusershow.add_trace(go.Scatter(y=dftrend_user_real.index, x=dftrend_user_real['User_Score']*10, text='User_Score',textposition='top center',mode='lines+markers', orientation = 'h',name='User_Score', marker_color='rgb(255, 0, 0)'))
  figcriticshow.update_xaxes(range=[50, 100])
  figusershow.show()

  dftrend_compare = dftrend 
  dftrend_compare = dftrend_compare.drop(dftrend_compare[dftrend_compare['User_Score'] == 'tbd'].index)
  dftrend_compare = dftrend_compare.dropna(subset=['User_Score'])
  dftrend_compare['User_Score'] = dftrend_compare['User_Score'].astype(float)
  dftrend_compare = dftrend_compare.groupby(['Year'])['User_Score','Critic_Score'].mean().round(2).reset_index()
  dftrend_compare

  figcompare = px.line(dftrend_compare, x="Year", y=["User_Score"], title='User Score Trend  by platforms',markers=True)
  figcompare.add_trace(go.Scatter(x=dftrend_compare['Year'], y=dftrend_compare['Critic_Score']/10, name='Critic_Score'))
  figcompare.update_traces(textposition="bottom right")
  figcompare.show()

  dfplatnumber = dftrend['Game_Platform'].value_counts(ascending=True).tail(10).reset_index()
  figplatnumber = px.bar(dfplatnumber, y="index", x="Game_Platform", title='Number of Games by Platforms',orientation='h',)
  figplatnumber.show()

  dfplatnum = dftrend.groupby(['Year','Game_Platform'])['Game_Platform'].count().reset_index(name='count')
  figplatnum = px.line(dfplatnum, x="Year", y="count", color='Game_Platform',title='Number of games released by platforms',markers=True)
  figplatnum.show()

  dftrend_platform = dftrend.groupby(['Game_Platform','Year']).mean().round(2).reset_index()
  dftrend_year = dftrend.groupby(['Year']).mean().round(2).reset_index()

  figplat = px.line(dftrend_platform, x="Year", y="Critic_Score", color='Game_Platform',title='Critic Score Trend by platforms',markers=True)
  figplat.update_traces(textposition="bottom right")
  figplat.update_yaxes(range=[30, 100])
  figplat.add_trace(go.Bar(
  x=dftrend_year['Year'], y=dftrend_year['Critic_Score'], text=dftrend_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplat.show()

  dftrend_platform_user = dftrend .drop(dftrend [dftrend ['User_Score'] == 'tbd'].index)
  dftrend_platform_user = dftrend_platform_user.dropna(subset=['User_Score'])
  dftrend_platform_user['User_Score'] = dftrend_platform_user['User_Score'].astype(float)
  dftrend_platform_user = dftrend_platform_user.groupby(['Game_Platform','Year'])['User_Score'].mean().round(2).reset_index()

  figplatuser = px.line(dftrend_platform_user, x="Year", y="User_Score", color='Game_Platform',title='User Score Trend by platforms',markers=True)
  figplatuser.update_traces(textposition="bottom right")
  figplatuser.update_yaxes(range=[0, 10])
  figplatuser.add_trace(go.Bar(
  x=dftrend_year['Year'], y=dftrend_year['Critic_Score']/10, text=dftrend_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplatuser.show()

  dftrend_genre = dftrend 
  dftrend_genre['Game_Genre'] = dftrend_genre['Game_Genre'].str.split('/')
  df2 = dftrend_genre.explode('Game_Genre').reset_index(drop=True)
  df2 = df2.dropna(subset=['Game_Genre'])

  dfnumgenre = df2.groupby(['Year','Game_Genre'])['Game_Genre'].count().reset_index(name='count')
  fignumgenre = px.line(dfnumgenre, x="Year", y="count", color='Game_Genre',title='Number of games released by genre',markers=True)
  fignumgenre.show()

  dfgenre = df2['Game_Genre'].value_counts(ascending=True).tail(10).reset_index()
  figgenrenumber = px.bar(dfgenre, y="index", x="Game_Genre", title='Number of Games by Genre',orientation='h',)
  figgenrenumber.show()

  top_genres = df2['Game_Genre'].value_counts().index[:10] #Select top 10 genres
  df2 = df2[df2['Game_Genre'].isin(top_genres)]

  dftrend_gen_user = df2.drop(df2[df2['User_Score'] == 'tbd'].index)
  dftrend_gen_user = dftrend_gen_user.dropna(subset=['User_Score'])
  dftrend_gen_user['User_Score'] = dftrend_gen_user['User_Score'].astype(float)

  dftrend_gen = df2.groupby(['Game_Genre','Year']).mean().round(2).reset_index()
  dftrend_gen_user = dftrend_gen_user.groupby(['Game_Genre','Year'])['User_Score'].mean().round(2).reset_index()

  figplatgen = px.line(dftrend_gen, x="Year", y="Critic_Score", color='Game_Genre',title='Critic Score Trend  by genres',markers=True)
  figplatgen.update_traces(textposition="bottom right")
  figplatgen.update_yaxes(range=[0, 100])
  figplatgen.add_trace(go.Bar(
  x=dftrend_year['Year'], y=dftrend_year['Critic_Score'], text=dftrend_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplatgen.show()

  figplatgen_user = px.line(dftrend_gen_user, x="Year", y="User_Score", color='Game_Genre',title='User Score Trend  by genres',markers=True)
  figplatgen_user.update_traces(textposition="bottom right")
  figplatgen_user.update_yaxes(range=[0, 10])
  figplatgen_user.add_trace(go.Bar(
  x=dftrend_year['Year'], y=dftrend_year['Critic_Score']/10, text=dftrend_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
  )
  figplatgen_user.show()


### **Specific Time Trend**

In [5]:
def specific_time():
    from datetime import datetime
    time = input(str("Enter the starting time and ending time in the format like xxxx-xx-xx, and split by comma: "))
    time = time.strip().replace(" ","").split(',')
    try:
        time_temp = [datetime.strptime(i, '%Y-%m-%d').date() for i in time]
        display(str(time_temp[0]))
        display(str(time_temp[1]))
    except:
        print('Fromat is not correct, please try again')

    dftime = df
    dftime['Game_Release_Date'] = pd.to_datetime(dftime['Game_Release_Date']).dt.date
    dftimespecific = dftime[(dftime['Game_Release_Date'] >= time_temp[0]) & (dftime['Game_Release_Date'] <= time_temp[1])].reset_index()

    dftimespecific['Month'] = dftimespecific['Game_Release_Date'].apply(lambda x: x.strftime("%Y-%m"))

    dftimespecific_criticscore = dftimespecific.groupby(['Game_Title'])['Critic_Score'].mean().round(2).reset_index()
    dftimespecific_criticscore.sort_values(by=['Critic_Score'], ascending=False, inplace=True)
    dftimespecific_critic_show = dftimespecific_criticscore.set_index('Game_Title')

    dftimespecific_userscore = dftimespecific.drop(dftimespecific[dftimespecific['User_Score'] == 'tbd'].index)
    dftimespecific_userscore = dftimespecific_userscore.dropna(subset=['User_Score'])
    dftimespecific_userscore['User_Score'] = dftimespecific_userscore['User_Score'].astype(float)
    dftimespecific_userscore = dftimespecific_userscore.groupby(['Game_Title'])['User_Score'].mean().round(2).reset_index()
    dftimespecific_userscore.sort_values(by=['User_Score'], ascending=False, inplace=True)
    dftimespecific_user_show = dftimespecific_userscore.set_index('Game_Title')

    dftimespecific_real = pd.merge(dftimespecific_critic_show, dftimespecific_user_show, on='Game_Title')
    dftimespecific_critic_real = dftimespecific_real.sort_values(by=['Critic_Score'], ascending=True).tail(10)
    dftimespecific_user_real = dftimespecific_real.sort_values(by=['User_Score'], ascending=True).tail(10)

    figcriticshow = px.bar(dftimespecific_critic_real, y=dftimespecific_critic_real.index, x="Critic_Score", title='Top 10 Games by Critic Score between '+str(time_temp[0])+' and '+str(time_temp[1]),orientation='h', text='Critic_Score', color_discrete_sequence=['#2E7D32'])
    figcriticshow.update_traces(texttemplate='%{text:.4s}', textposition='outside')
    figcriticshow.add_trace(go.Scatter(y=dftimespecific_critic_real.index, x=dftimespecific_critic_real['User_Score']*10, text='User_Score',mode='lines+markers', orientation = 'h',name='User_Score', marker_color='#FFA000'))
    figcriticshow.update_xaxes(range=[50, 100])
    figcriticshow.show()

    figusershow = px.bar(dftimespecific_user_real, y=dftimespecific_user_real.index, x="Critic_Score", title='Top 10 Games by User Score between '+str(time_temp[0])+' and '+str(time_temp[1]),orientation='h', color_discrete_sequence=['rgb(179, 229, 252)'])
    figusershow.add_trace(go.Scatter(y=dftimespecific_user_real.index, x=dftimespecific_user_real['User_Score']*10, text='User_Score',textposition='top center',mode='lines+markers', orientation = 'h',name='User_Score', marker_color='rgb(255, 0, 0)'))
    figcriticshow.update_xaxes(range=[50, 100])
    figusershow.show()

    dftimespecific_compare = dftimespecific
    dftimespecific_compare = dftimespecific_compare.drop(dftimespecific_compare[dftimespecific_compare['User_Score'] == 'tbd'].index)
    dftimespecific_compare = dftimespecific_compare.dropna(subset=['User_Score'])
    dftimespecific_compare['User_Score'] = dftimespecific_compare['User_Score'].astype(float)
    dftimespecific_compare = dftimespecific_compare.groupby(['Month'])['User_Score','Critic_Score'].mean().round(2).reset_index()
    dftimespecific_compare

    figcompare = px.line(dftimespecific_compare, x="Month", y=["User_Score"], title='User Score Trend between '+str(time_temp[0])+' and '+str(time_temp[1])+' by platforms',markers=True)
    figcompare.add_trace(go.Scatter(x=dftimespecific_compare['Month'], y=dftimespecific_compare['Critic_Score']/10, name='Critic_Score'))
    figcompare.update_traces(textposition="bottom right")
    figcompare.show()

    dfplatnumber = dftimespecific['Game_Platform'].value_counts(ascending=True).tail(10).reset_index()
    figplatnumber = px.bar(dfplatnumber, y="index", x="Game_Platform", title='Number of Games by Platforms',orientation='h',)
    figplatnumber.show()

    dfplatnum = dftimespecific.groupby(['Month','Game_Platform'])['Game_Platform'].count().reset_index(name='count')
    figplatnum = px.line(dfplatnum, x="Month", y="count", color='Game_Platform',title='Number of games released between '+str(time_temp[0])+' and '+str(time_temp[1])+' by platforms',markers=True)
    figplatnum.show()

    dftimespecific_platform = dftimespecific.groupby(['Game_Platform','Month']).mean().round(2).reset_index()
    dftimespecific_year = dftimespecific.groupby(['Month']).mean().round(2).reset_index()

    figplat = px.line(dftimespecific_platform, x="Month", y="Critic_Score", color='Game_Platform',title='Critic Score Trend between '+str(time_temp[0])+' and '+str(time_temp[1])+' by platforms',markers=True)
    figplat.update_traces(textposition="bottom right")
    figplat.update_yaxes(range=[30, 100])
    figplat.add_trace(go.Bar(
    x=dftimespecific_year['Month'], y=dftimespecific_year['Critic_Score'], text=dftimespecific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
    )
    figplat.show()

    dftimespecific_platform_user = dftimespecific.drop(dftimespecific[dftimespecific['User_Score'] == 'tbd'].index)
    dftimespecific_platform_user = dftimespecific_platform_user.dropna(subset=['User_Score'])
    dftimespecific_platform_user['User_Score'] = dftimespecific_platform_user['User_Score'].astype(float)
    dftimespecific_platform_user = dftimespecific_platform_user.groupby(['Game_Platform','Month'])['User_Score'].mean().round(2).reset_index()

    figplatuser = px.line(dftimespecific_platform_user, x="Month", y="User_Score", color='Game_Platform',title='User Score Trend between '+str(time_temp[0])+' and '+str(time_temp[1])+' by platforms',markers=True)
    figplatuser.update_traces(textposition="bottom right")
    figplatuser.update_yaxes(range=[0, 10])
    figplatuser.add_trace(go.Bar(
    x=dftimespecific_year['Month'], y=dftimespecific_year['Critic_Score']/10, text=dftimespecific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
    )
    figplatuser.show()

    dftimespecific_genre = dftimespecific
    dftimespecific_genre['Game_Genre'] = dftimespecific_genre['Game_Genre'].str.split('/')
    df2 = dftimespecific_genre.explode('Game_Genre').reset_index(drop=True)
    df2 = df2.dropna(subset=['Game_Genre'])

    dfnumgenre = df2.groupby(['Month','Game_Genre'])['Game_Genre'].count().reset_index(name='count')
    fignumgenre = px.line(dfnumgenre, x="Month", y="count", color='Game_Genre',title='Number of games released between '+str(time_temp[0])+' and '+str(time_temp[1])+' by genre',markers=True)
    fignumgenre.show()

    dfgenre = df2['Game_Genre'].value_counts(ascending=True).tail(10).reset_index()
    figgenrenumber = px.bar(dfgenre, y="index", x="Game_Genre", title='Number of Games released between '+str(time_temp[0])+' and '+str(time_temp[1])+' by Genre',orientation='h',)
    figgenrenumber.show()

    top_genres = df2['Game_Genre'].value_counts().index[:10] #Select top 10 genres
    df2 = df2[df2['Game_Genre'].isin(top_genres)]

    dftimespecific_gen_user = df2.drop(df2[df2['User_Score'] == 'tbd'].index)
    dftimespecific_gen_user = dftimespecific_gen_user.dropna(subset=['User_Score'])
    dftimespecific_gen_user['User_Score'] = dftimespecific_gen_user['User_Score'].astype(float)

    dftimespecific_gen = df2.groupby(['Game_Genre','Month']).mean().round(2).reset_index()
    dftimespecific_gen_user = dftimespecific_gen_user.groupby(['Game_Genre','Month'])['User_Score'].mean().round(2).reset_index()

    figplatgen = px.line(dftimespecific_gen, x="Month", y="Critic_Score", color='Game_Genre',title='Critic Score Trend between '+str(time_temp[0])+' and '+str(time_temp[1])+' by genres',markers=True)
    figplatgen.update_traces(textposition="bottom right")
    figplatgen.update_yaxes(range=[0, 100])
    figplatgen.add_trace(go.Bar(
    x=dftimespecific_year['Month'], y=dftimespecific_year['Critic_Score'], text=dftimespecific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
    )
    figplatgen.show()

    figplatgen_user = px.line(dftimespecific_gen_user, x="Month", y="User_Score", color='Game_Genre',title='User Score Trend between '+str(time_temp[0])+' and '+str(time_temp[1])+' by genres',markers=True)
    figplatgen_user.update_traces(textposition="bottom right")
    figplatgen_user.update_yaxes(range=[0, 10])
    figplatgen_user.add_trace(go.Bar(
    x=dftimespecific_year['Month'], y=dftimespecific_year['Critic_Score']/10, text=dftimespecific_year['Critic_Score'], textposition='auto', name='Critic_Score',marker_color='rgb(179, 229, 252)')
    )
    figplatgen_user.show()    

## **TimePerdiction**

In [6]:
#tsfresh
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute, make_forecasting_frame
from tsfresh.feature_extraction import ComprehensiveFCParameters, settings

In [7]:
#tsfresh
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute, make_forecasting_frame
from tsfresh.feature_extraction import ComprehensiveFCParameters, settings

covidtime = pd.Timestamp('2020-01-01')
covidendtime = pd.Timestamp('2022-12-31')

dfprecovid = df[df['Game_Release_Date'] < covidtime]
dfcovid = df.loc[(df['Game_Release_Date'] >= covidtime) & (df['Game_Release_Date'] <= covidendtime)]
dfaftercovid = df[df['Game_Release_Date'] > covidendtime]

##### **All Time**

In [ ]:
#autots
from autots import AutoTS
modelalltime = AutoTS(frequency='infer', ensemble='simple', drop_data_older_than_periods=100)
modelalltime = modelalltime.fit(df, date_col='Game_Release_Date', value_col='Critic_Score', id_col=None)
modelalltime.export_template(
"modelalltime.csv",
models="best",
max_per_model_class=1,
include_results=True,)

In [9]:
modelalltime.import_template("modelalltime.csv",method="only",enforce_model_list=True)

predictionall = modelalltime.predict(forecast_length=45)
forecastall = predictionall.forecast
forecastall.reset_index()

dftimeper=df.drop(df[df['Year'] == 2005].index)
dftimeper = dftimeper[['Critic_Score','Game_Release_Date']]
dftimeper = dftimeper.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

figtimeallpre = px.line(dftimeper, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend of ALL TIME',markers=True)
figtimeallpre.update_traces(textposition="bottom right")
figtimeallpre.add_trace(go.Scatter(x=forecastall.index, y=forecastall["Critic_Score"], name='Critic_Score',marker_color='rgb(179, 229, 252)'))
figtimeallpre.show()

##### **Before covid**

In [ ]:
modelprecovid = AutoTS(frequency='infer', ensemble='simple', drop_data_older_than_periods=100)
modelprecovid = modelprecovid.fit(dfprecovid, date_col='Game_Release_Date', value_col='Critic_Score', id_col=None)
modelprecovid.export_template(
"modelprecovid.csv",
models="best",
max_per_model_class=1,
include_results=True,)

In [11]:
modelprecovid.import_template(
"modelalltime.csv",
method="only",
enforce_model_list=True)
predictionprecovid = modelprecovid.predict(forecast_length=45)
forecastprecovid = predictionprecovid.forecast
forecastprecovid.reset_index()

dftimepercovid = dfprecovid[['Critic_Score','Game_Release_Date']]
dftimepercovid = dftimepercovid.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

figtimeallprecovid = px.line(dftimepercovid, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend before COVID',markers=True)
figtimeallprecovid.update_traces(textposition="bottom right")
figtimeallprecovid.add_trace(go.Scatter(x=forecastprecovid.index, y=forecastprecovid["Critic_Score"], name='Perdicited_Critic_Score',marker_color='rgb(179, 229, 252)'))
figtimeallprecovid.show()

c:\ProgramData\Anaconda3\envs\kidul\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



##### **During Covid**

In [ ]:
modelcovid = AutoTS(frequency='infer', ensemble='simple', drop_data_older_than_periods=100)
modelcovid = modelcovid.fit(dfcovid, date_col='Game_Release_Date', value_col='Critic_Score', id_col=None)
modelcovid.export_template(
"modelcovid.csv",
models="best",
max_per_model_class=1,
include_results=True,)

In [17]:
modelcovid.import_template(
"modelcovid.csv",
method="only",
enforce_model_list=True)

predictioncovid = modelcovid.predict(forecast_length=45)
forecastcovid = predictioncovid.forecast
forecastcovid.reset_index()

dftimecovid = dfcovid[['Critic_Score','Game_Release_Date']]
dftimecovid = dftimecovid.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

figtimeallcovid = px.line(dftimecovid, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend before COVID',markers=True)
figtimeallcovid.update_traces(textposition="bottom right")
figtimeallcovid.add_trace(go.Scatter(x=forecastcovid.index, y=forecastcovid["Critic_Score"], name='Critic_Score',marker_color='rgb(179, 229, 252)'))
figtimeallcovid.show()

##### **After covid**

In [ ]:
modelaftercovid = AutoTS(frequency='infer', ensemble='simple', drop_data_older_than_periods=100)
modelaftercovid = modelaftercovid.fit(dfaftercovid, date_col='Game_Release_Date', value_col='Critic_Score', id_col=None)
modelaftercovid.export_template(
"modelaftercovid.csv",
models="best",
max_per_model_class=1,
include_results=True,)

In [18]:
modelaftercovid.import_template(
"modelaftercovid.csv",
method="only",
enforce_model_list=True)

predictionaftercovid = modelaftercovid.predict(forecast_length=45)
forecastaftercovid = predictionaftercovid.forecast
forecastaftercovid.reset_index()

dftimeaftercovid = dfaftercovid[['Critic_Score','Game_Release_Date']]
dftimeaftercovid = dftimeaftercovid.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

figtimeaftercovid = px.line(dftimeaftercovid, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend before COVID',markers=True)
figtimeaftercovid.update_traces(textposition="bottom right")
figtimeaftercovid.add_trace(go.Scatter(x=forecastaftercovid.index, y=forecastaftercovid["Critic_Score"], name='Critic_Score',marker_color='rgb(179, 229, 252)'))
figtimeaftercovid.show()

In [32]:
forecastall = forecastall.reset_index()
forecastaftercovid = forecastaftercovid.reset_index()
forecastcovid=forecastcovid.reset_index()
forecastprecovid=forecastprecovid.reset_index()

fig_all_perdict = px.line(forecastall, x=forecastall.index, y="Critic_Score", title='Critic Score Perdiction of ALL TIME',markers=True)
fig_all_perdict.add_trace(go.Scatter(x=forecastprecovid.index, y=forecastprecovid["Critic_Score"], name='Pre-COVID'))
fig_all_perdict.add_trace(go.Scatter(x=forecastcovid.index, y=forecastcovid["Critic_Score"], name='During COVID'))
fig_all_perdict.add_trace(go.Scatter(x=forecastaftercovid.index, y=forecastaftercovid["Critic_Score"], name='After-COVID'))
fig_all_perdict.update_traces(textposition="bottom right")
fig_all_perdict.show()

In [33]:
def perdiction_time():
    #tsfresh
    from tsfresh import extract_features, extract_relevant_features, select_features
    from tsfresh.utilities.dataframe_functions import impute, make_forecasting_frame
    from tsfresh.feature_extraction import ComprehensiveFCParameters, settings

    covidtime = pd.Timestamp('2020-01-01')
    covidendtime = pd.Timestamp('2022-12-31')

    dfprecovid = df[df['Game_Release_Date'] < covidtime]
    dfcovid = df.loc[(df['Game_Release_Date'] >= covidtime) & (df['Game_Release_Date'] <= covidendtime)]
    dfaftercovid = df[df['Game_Release_Date'] > covidendtime]

    modelalltime.import_template("modelalltime.csv",method="only",enforce_model_list=True)

    predictionall = modelalltime.predict(forecast_length=45)
    forecastall = predictionall.forecast
    forecastall.reset_index()

    dftimeper=df.drop(df[df['Year'] == 2005].index)
    dftimeper = dftimeper[['Critic_Score','Game_Release_Date']]
    dftimeper = dftimeper.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

    figtimeallpre = px.line(dftimeper, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend of ALL TIME',markers=True)
    figtimeallpre.update_traces(textposition="bottom right")
    figtimeallpre.add_trace(go.Scatter(x=forecastall.index, y=forecastall["Critic_Score"], name='Critic_Score',marker_color='rgb(179, 229, 252)'))
    figtimeallpre.show()

    modelprecovid.import_template(
    "modelalltime.csv",
    method="only",
    enforce_model_list=True)
    predictionprecovid = modelprecovid.predict(forecast_length=45)
    forecastprecovid = predictionprecovid.forecast
    forecastprecovid.reset_index()

    dftimepercovid = dfprecovid[['Critic_Score','Game_Release_Date']]
    dftimepercovid = dftimepercovid.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

    figtimeallprecovid = px.line(dftimepercovid, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend before COVID',markers=True)
    figtimeallprecovid.update_traces(textposition="bottom right")
    figtimeallprecovid.add_trace(go.Scatter(x=forecastprecovid.index, y=forecastprecovid["Critic_Score"], name='Perdicited_Critic_Score',marker_color='rgb(179, 229, 252)'))
    figtimeallprecovid.show()

    modelcovid.import_template(
    "modelcovid.csv",
    method="only",
    enforce_model_list=True)

    predictioncovid = modelcovid.predict(forecast_length=45)
    forecastcovid = predictioncovid.forecast
    forecastcovid.reset_index()

    dftimecovid = dfcovid[['Critic_Score','Game_Release_Date']]
    dftimecovid = dftimecovid.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

    figtimeallcovid = px.line(dftimecovid, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend before COVID',markers=True)
    figtimeallcovid.update_traces(textposition="bottom right")
    figtimeallcovid.add_trace(go.Scatter(x=forecastcovid.index, y=forecastcovid["Critic_Score"], name='Critic_Score',marker_color='rgb(179, 229, 252)'))
    figtimeallcovid.show()

    modelaftercovid.import_template(
    "modelaftercovid.csv",
    method="only",
    enforce_model_list=True)

    predictionaftercovid = modelaftercovid.predict(forecast_length=45)
    forecastaftercovid = predictionaftercovid.forecast
    forecastaftercovid.reset_index()

    dftimeaftercovid = dfaftercovid[['Critic_Score','Game_Release_Date']]
    dftimeaftercovid = dftimeaftercovid.groupby(['Game_Release_Date'])['Critic_Score'].mean().reset_index()

    figtimeaftercovid = px.line(dftimeaftercovid, x="Game_Release_Date", y="Critic_Score", title='Critic Score Trend before COVID',markers=True)
    figtimeaftercovid.update_traces(textposition="bottom right")
    figtimeaftercovid.add_trace(go.Scatter(x=forecastaftercovid.index, y=forecastaftercovid["Critic_Score"], name='Critic_Score',marker_color='rgb(179, 229, 252)'))
    figtimeaftercovid.show()

    forecastall = forecastall.reset_index()
    forecastaftercovid = forecastaftercovid.reset_index()
    forecastcovid=forecastcovid.reset_index()
    forecastprecovid=forecastprecovid.reset_index()

    fig_all_perdict = px.line(forecastall, x=forecastall.index, y="Critic_Score", title='Critic Score Perdiction of ALL TIME',markers=True)
    fig_all_perdict.add_trace(go.Scatter(x=forecastprecovid.index, y=forecastprecovid["Critic_Score"], name='Pre-COVID'))
    fig_all_perdict.add_trace(go.Scatter(x=forecastcovid.index, y=forecastcovid["Critic_Score"], name='During COVID'))
    fig_all_perdict.add_trace(go.Scatter(x=forecastaftercovid.index, y=forecastaftercovid["Critic_Score"], name='After-COVID'))
    fig_all_perdict.update_traces(textposition="bottom right")
    fig_all_perdict.show()


## **All combined**

In [16]:
def showOptions():
    choices = ["All time Summary", "Select specific publisher","Select specific period", "Perdiction","exit"]
    print("Please choose following option:")
    print("1. ", choices[0])
    print("2. ", choices[1])
    print("3. ", choices[2])
    print("4. ", choices[3])
    print("5. ", choices[4])

    choice = input('And you are going to : ')
    # with conn:
    while True:
        if choice == '5':
            print("© AIDM-7350 The Lord of Sham Shui Po") 
            break
        elif choice == '1':
            alltimetrend()
            returnOptions()
            break
        elif choice == '2':
            get_specific()
            returnOptions()
            break
        elif choice == '3':
            specific_time()
            returnOptions()
            break
        elif choice == '4':
            perdiction_time()
            break
        else:
            print("This option is not available")
            continue      

In [34]:
showOptions()

Please choose following option:
1.  All time Summary
2.  Select specific publisher
3.  Select specific period
4.  Perdiction
5.  exit


c:\ProgramData\Anaconda3\envs\kidul\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



## **Presentation Example**

In [19]:
alltimetrend()

In [35]:
get_specific()

,Game_Publisher
1,Electronic Arts
2,Skookum Arts
3,Skookum Arts LLC


Electronic Arts


In [38]:
specific_time()

'2019-12-31'

'2023-01-01'